In [1]:
import torchvision.models as models
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
import foolbox.attacks as fa

from torchvision import datasets, transforms

import torchvision
import torchvision.transforms as transforms

from models.VGG16_with_flex_v5 import *

epsilons = [0.0, 0.03, 0.1, 0.3]
#epsilons = [0, .3]

# ======================================== prepare the dataset ==========================================================================================
mean_cifar10 = [0.485, 0.456, 0.406]   
std_cifar10 = [0.229, 0.224, 0.225]
batch_size = 1

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10,std_cifar10),
])

testset = torchvision.datasets.CIFAR10(root='../FlexibleCNNs/data', train=False, download=False, transform=transform_test)

sampler_test = list(range(0, len(testset), 100))
testset_samp = torch.utils.data.Subset(testset, sampler_test)

testloader = torch.utils.data.DataLoader(testset_samp, batch_size=100, shuffle=True, num_workers=2)


model = VGG16().to("cuda")
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = PyTorchModel(model, bounds=(-2.117, 2.64), preprocessing=preprocessing)

# Load the pretrained model
state = torch.load('./models/VGG16-flex-v5-block1-model_150_90.pth')
model.load_state_dict(state['model']) 

model.eval()


with torch.no_grad():
    for data in testloader:
        images, labels = data
        # get data and test the model
        # wrapping the tensors with ep.astensors is optional, but it allows
        # us to work with EagerPy tensors in the following
        print(accuracy(fmodel, images.cuda(), labels.cuda()))

        # apply the attack
        attack = fa.boundary_attack.BoundaryAttack(
        steps=2500,
        )

        advs, _, success = attack(fmodel, images.cuda(), labels.cuda(), epsilons=epsilons)
        
        
        images = torch.stack(advs)
        images = torch.reshape(images[0,:], (100, 3, 32, 32))
        
        outputs = model(images.cuda())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum().item()

        print('0.0 Accuracy on the test dataset is: %f %%' % (
            100 * correct / total))

/home/mlavrov/anaconda3/envs/meng/lib/python3.7/site-packages/foolbox/models/pytorch.py:37: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  "The PyTorch model is in training mode and therefore might"


0.8100000023841858


NameError: name 'total' is not defined

torch.Size([1000, 3, 32, 32])